In [251]:
import pandas as pd #data manipulation and analysis
import numpy as np #data arrays
from datetime import datetime #date time
import glob #join csv's together
import os # makes file join OS independent
import plotly.express as px #data visualisation
from geopy import distance

In [252]:
#load downloaded csv in, change date strings to datetime64 (date/time format)
#define variables that we will use for concatenating
path =r'C:\Users\zaydm\Documents\Repos\Google_Data_Analytics_Capstone\CSV files'
file_type = 'csv'
seperator =','

#define our dataframe. Change date strings to datetime64 
#'infer_datetime_format' infers the format of the datetime strings - in this case parsing speed = 10x faster
bike_share = pd.concat([pd.read_csv(f, sep=seperator,parse_dates=['started_at', 'ended_at'],infer_datetime_format=True, dayfirst=True) 

#using glob/for loop to find the csv files from the specified folder to join them together
for f in glob.glob(os.path.join(path + "/*."+file_type))],ignore_index=True)

#rename datetime columns so they're better defined
bike_share.rename(columns= {'started_at' : 'start_time','ended_at' : 'end_time' }, inplace=True)

In [253]:
bike_share

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219374,211BE0DC162D85B7,electric_bike,2022-02-23 17:47:00,2022-02-23 18:02:00,NaN,NaN,Leavitt St & Chicago Ave,18058,41.880000,-87.630000,41.895501,-87.682017,member
219375,D4D53E78000C8CA1,electric_bike,2022-02-04 10:43:00,2022-02-04 10:50:00,NaN,NaN,Leavitt St & Chicago Ave,18058,41.910000,-87.680000,41.895501,-87.682017,member
219376,9E85F07D2F94492B,electric_bike,2022-02-28 09:16:00,2022-02-28 09:28:00,Wood St & Chicago Ave,637,Canal St & Adams St,13011,41.895714,-87.672209,41.879255,-87.639904,member
219377,B61B559F81F1D823,electric_bike,2022-02-10 16:55:00,2022-02-10 16:57:00,NaN,NaN,Canal St & Adams St,13011,41.880000,-87.630000,41.879255,-87.639904,member


In [254]:
#create new column that displays ride length
bike_share['ride_length_seconds'] = bike_share['end_time'] - bike_share['start_time']

#create ride length in seconds for analysis. The error below is false positive 
bike_share['ride_length_seconds'] = bike_share['ride_length_seconds'].dt.total_seconds()

In [255]:
bike_share.head(5)

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length_seconds
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual,177.0
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual,261.0
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member,261.0
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual,896.0
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member,362.0


In [256]:
#get day of week (dow) integer for the dates in "start time" column
bike_share['day_of_week'] = bike_share['start_time'].dt.dayofweek

#replace the integer values of days of week to actual text using a dictionary
#days = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thur', 4:'Fri', 5:'Sat', 6:'Sun'}
days = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7}
bike_share['day_of_week'] = bike_share['day_of_week'].replace(days)

In [257]:
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length_seconds,day_of_week
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual,177.0,4
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual,261.0,1
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member,261.0,2
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual,896.0,2
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member,362.0,4


In [258]:
missing_values_count = bike_share.isnull().sum()

missing_values_count[:]
#We can see from start_station_id there are 34840 missing values and from end_station_id there are 38282 missing values

ride_id                    0
rideable_type              0
start_time                 0
end_time                   0
start_station_name     34840
start_station_id       34840
end_station_name       38282
end_station_id         38282
start_lat                  0
start_lng                  0
end_lat                  163
end_lng                  163
member_casual              0
ride_length_seconds        0
day_of_week                0
dtype: int64

In [259]:
#since we can't create start/end station names, we can drop the entire rows using dropna()
bike_share = bike_share.dropna()

#second check ensuring we have no null values
missing_values_count_2 = bike_share.isnull().sum()
missing_values_count_2[:]

ride_id                0
rideable_type          0
start_time             0
end_time               0
start_station_name     0
start_station_id       0
end_station_name       0
end_station_id         0
start_lat              0
start_lng              0
end_lat                0
end_lng                0
member_casual          0
ride_length_seconds    0
day_of_week            0
dtype: int64

In [260]:
#checking to ensure no duplicate ride_id
unique_id = len(pd.unique(bike_share["ride_id"]))

print("# of unique Id: " + str(unique_id))

# of unique Id: 169306


In [261]:
#checking for any negative ride lengths as previous year data had some errors
#neg_time = bike_share[bike_share['ride_length_seconds'] < 0]
#print(neg_time)

After realising we have both lat and long start and finish points, we can work out the distance between the start and finish times. Its important to note that whilst these wont tell us the exact distance of the rides as it'll work out the path in a straight line and ofcourse won't include roads,obstructions etc. The purpose of this calculation is to see how much riders travel relative to each other and then compare volumes too. For e.g. member rides ride on average 7 miles on each trip, casual riders ride 10 miles on each trip. We then see volume on weekdays - this could indicate that member riders are commuting - faster speeds, same distance whereas the casual riders are riding more on weekends, slower speeds but longer distances. 

In [262]:
index_0_start = (42.012800,	-87.665906)
index_0_end = (42.012560,	-87.674367)
print(distance.distance(index_0_start, index_0_end).miles)

0.43580980480248127


In [263]:
#change latitude and longitude columns from string to float
bike_share.astype({'start_lat':'float', 'start_lng':'float', 'end_lat':'float', 'end_lng':'float'})
bike_share.dtypes

ride_id                        object
rideable_type                  object
start_time             datetime64[ns]
end_time               datetime64[ns]
start_station_name             object
start_station_id               object
end_station_name               object
end_station_id                 object
start_lat                     float64
start_lng                     float64
end_lat                       float64
end_lng                       float64
member_casual                  object
ride_length_seconds           float64
day_of_week                     int64
dtype: object

In [264]:
#Joining latitude and longitude in a single column to provide a start/end location
bike_share['start_lat_long'] = [', '.join(str(x) for x in y) for y in map(tuple, bike_share[['start_lat', 'start_lng']].values)]
bike_share['end_lat_long'] = [', '.join(str(x) for x in y) for y in map(tuple, bike_share[['end_lat', 'end_lng']].values)]

C:\Users\zaydm\miniconda3\envs\crypto_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\zaydm\miniconda3\envs\crypto_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [265]:
index_0_start = ('41.89571428, -87.6722095')
index_0_end = ('41.879255, -87.639904')
print(distance.distance(index_0_start, index_0_end).miles)
#Vincenty distance (vincenty) uses a more accurate ellipsoidal model of the earth.

2.016462858476188


In [266]:
# Geopy can calculate geodesic distance between two points. The geodesic distance is the shortest distance on the surface of an ellipsoidal model of the earth. 
# The default algorithm uses the method is given by Karney (2013) (geodesic); this is accurate to round-off and always converges.
# To allow for it to work for columns, we define distancer and use row to allow each ID's distance to be calculated and then placed into the journey distance column
# Apply does take some time to execute
# Uk units = miles
def distancer(row):
    coords_1 = (row['start_lat_long'])
    coords_2 = (row['end_lat_long'])
    return distance.distance(coords_1, coords_2).miles

bike_share['journey_distance_miles'] = bike_share.apply(distancer, axis=1)

C:\Users\zaydm\miniconda3\envs\crypto_env\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [267]:
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length_seconds,day_of_week,start_lat_long,end_lat_long,journey_distance_miles
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual,177.0,4,"42.0128005, -87.665906","42.01256011541, -87.6743671152",0.435817
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual,261.0,1,"42.012763, -87.6659675","42.01256011541, -87.6743671152",0.432562
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member,261.0,2,"41.9256018819, -87.6537080423","41.92533, -87.6658",0.623515
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual,896.0,2,"41.983593, -87.669154","41.961507, -87.671387",1.528651
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member,362.0,4,"41.87785, -87.62408","41.88462107257936, -87.62783423066139",0.505842


In [268]:
#drop columns we no longer need - cleaner DF
bike_share.drop(columns= ['start_lat', 'start_lng','end_lat', 'end_lng'], inplace=True)

C:\Users\zaydm\miniconda3\envs\crypto_env\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [269]:
bike_share.head()

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,ride_length_seconds,day_of_week,start_lat_long,end_lat_long,journey_distance_miles
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,casual,177.0,4,"42.0128005, -87.665906","42.01256011541, -87.6743671152",0.435817
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,casual,261.0,1,"42.012763, -87.6659675","42.01256011541, -87.6743671152",0.432562
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,member,261.0,2,"41.9256018819, -87.6537080423","41.92533, -87.6658",0.623515
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,casual,896.0,2,"41.983593, -87.669154","41.961507, -87.671387",1.528651
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,member,362.0,4,"41.87785, -87.62408","41.88462107257936, -87.62783423066139",0.505842


In [270]:
mc_type = bike_share.groupby('member_casual')
mc_type['ride_length_seconds'].mean()

member_casual
casual    1558.570363
member     627.947491
Name: ride_length_seconds, dtype: float64

In [281]:
mc_day_type = bike_share.groupby(['member_casual','day_of_week'])

bike_share_mean = mc_day_type['ride_length_seconds'].mean().reset_index()



In [284]:
weekday = {1:'Mon', 2:'Tue', 3:'Wed', 4:'Thur', 5:'Fri', 6:'Sat', 7:'Sun'}

bike_share_mean['day_of_week'] = bike_share_mean['day_of_week'].replace(weekday)

bike_share_mean.set_index('member_casual', inplace = True)

bike_share_mean

,day_of_week,ride_length_seconds
member_casual,,
casual,Mon,1454.452181
casual,Tue,1283.577874
casual,Wed,1555.046018
casual,Thur,1887.585099
casual,Fri,1246.290368
casual,Sat,1643.777397
casual,Sun,1799.109020
member,Mon,632.176459
member,Tue,607.971865


In [ ]:
volume_distance = bike_share.groupby('day_of_week')